### Chebyshev

The Chebyshev approximation problem is (for $i = 1,...,k$)
\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \max_i \left|b_i^Tx - c_i\right| \\
  \end{aligned}
\end{equation*}

with variable $x \in \mathbb{R}^{n}$, and constants $b_1,..., b_m \in \mathbb{R}^n$ and $c_1,...,c_m \in \mathbb{R}$.

We add an $\ell_2$ regularization term to form the problem:

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \max_i \| A^{(i)}x \|_2 + \left|b_i^Tx - c_i\right| \\
  \end{aligned}
\end{equation*}

or equivalently:

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \max_i [t_i + \left|b_i^Tx - c_i\right| ]\\
    &\text{subject to} && \|A^{(i)}x\|_2 \leq t_i &&&i = 1,...,k
  \end{aligned}
\end{equation*}

In [3]:
import cvxpy as cp
import numpy as np
import scipy as sp

# Variable declarations

problemID = "chebyshev_0"
prob = None
opt_val = None

def normalized_data_matrix(m, n, mu):
    if mu == 1:
        # dense
        A = np.random.randn(m, n)
        A /= np.sqrt(np.sum(A**2, 0))
    else:
        # sparse
        A = sp.rand(m, n, mu)
        A.data = np.random.randn(A.nnz)
        N = A.copy()
        N.data = N.data**2
        A = A*sp.diags([1 / np.sqrt(np.ravel(N.sum(axis=0)))], [0])

    return A

np.random.seed(0)
m = 500
n = 1000
k = 10
A = [normalized_data_matrix(m,n,1) for i in range(k)]
B = normalized_data_matrix(k,n,1)
c = np.random.rand(k)

# Problem 1 (Epigraph form)
x1 = cp.Variable(n)
t = cp.Variable(k)

f = cp.max_entries(t+cp.abs(B*x1-c))
C = []
for i in range(k):
    C.append(cp.pnorm(A[i]*x1, 2) <= t[i])

prob1 = cp.Problem(cp.Minimize(f), C)

# Problem 2 (Unconstrained)
x2 = cp.Variable(n)
obj_list = [cp.pnorm(A[i]*x2, 2) + cp.abs(B[i,:]*x2 - c[i]) for i in range(k)]
f2 = cp.max_elemwise(obj_list)

prob2 = cp.Problem(cp.Minimize(f2))

# Problem collection

# Single problem collection
problem1Dict = {
    "problemID" : "chebyshev_0",
    "problem"   : prob1,
    "opt_val"   : None
}
problem2Dict = {
    "problemID" : "chebyshev_epigraph_0",
    "problem"   : prob2,
    "opt_val"   : None
}
problems = [problem1Dict, problem2Dict]

# For debugging individual problems:
if __name__ == "__main__":
    def printResults(problemID = "", problem = None, opt_val = None):
        print(problemID)
        problem.solve(solver="superscs", verbose=2, eps=True, memory=100)
        print("\tstatus: {}".format(problem.status))
        print("\toptimal value: {}".format(problem.value))
        print("\ttrue optimal value: {}".format(opt_val))
    printResults(**problems[0])
    printResults(**problems[1])


chebyshev_0


SolverError: Unknown solver.